# Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import zipfile
import json
import dask.dataframe as dd

In [2]:
def load_data_year(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    meta = {'TP_ESTADO_CIVIL': 'i', 'NU_NOTA_COMP1': 'i', 'NU_NOTA_COMP2': 'i', 
            'NU_NOTA_COMP3': 'i', 'NU_NOTA_COMP4': 'i', 'NU_NOTA_COMP5': 'i'}
    try:
        df = dd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, assume_missing=True).head(n=1000)
    except:
        try:
            df = dd.read_csv(file, encoding='cp1252', usecols=cols, assume_missing=True).head(n=1000)
        except:
            df = dd.read_csv(file, encoding='cp1252', assume_missing=True).head(n=1000)
            df = df.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  
                                    'NOTA_MT': 'NU_NOTA_MT','IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IDADE': 'NU_IDADE',
                                    'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
                                    'UF_RESIDENCIA': 'SG_UF_RESIDENCIA',  'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
                                     'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})[cols]
    
    return df

In [3]:
cols_used = ['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
             'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
             'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
             'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
             'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
             'NU_NOTA_MT', 'NU_NOTA_COMP1',
             'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
             'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
             'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
             'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
             'Q023', 'Q024', 'Q025']

In [4]:
years = [2015, 2016, 2017, 2018, 2019]

In [5]:
def union_datas(years, cols):
    df_final = load_data_year(years[0], cols)
    for y in years[1:]:
        df_final = dd.concat([df_final, load_data_year(y, cols)])
    return df_final

In [6]:
%%time

df_union = union_datas(years, cols_used)

Wall time: 3.54 s


In [7]:
df_union.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ESTADO_CIVIL,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,1.500000e+11,2015.0,Gravataí,RS,42.0,M,1.0,10.0,1.0,1.0,...,B,A,A,B,A,A,B,B,B,B
1,1.500000e+11,2015.0,Montes Claros,MG,21.0,M,1.0,5.0,1.0,0.0,...,A,A,A,B,B,A,D,B,B,B
2,1.500000e+11,2015.0,Olinda,PE,22.0,M,4.0,0.0,1.0,1.0,...,A,A,A,A,A,A,C,B,B,A
3,1.500000e+11,2015.0,Belo Horizonte,MG,23.0,F,1.0,6.0,1.0,0.0,...,B,A,A,C,A,B,D,B,B,B
4,1.500000e+11,2015.0,Teófilo Otoni,MG,18.0,M,1.0,2.0,1.0,0.0,...,A,A,B,B,A,A,C,B,B,B


# Pre-processing

In [8]:
pd.DataFrame({'types': df_union.dtypes, 'missing': df_union.isna().compute().sum()})

,types,missing
NU_INSCRICAO,float64,0
NU_ANO,float64,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,float64,0
TP_SEXO,object,0
TP_ST_CONCLUSAO,float64,0
TP_ANO_CONCLUIU,float64,0
TP_ESCOLA,float64,0
TP_ESTADO_CIVIL,float64,177


In [9]:
df_final = df_union

In [10]:
type(df_final)

dask.dataframe.core.DataFrame

In [11]:
df_final = df_final.replace(np.nan, -1)

In [12]:
cols_int = ['TP_ESTADO_CIVIL', 'NU_NOTA_COMP1','NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4','NU_NOTA_COMP5']
df_final[cols_int] = df_final[cols_int].astype('i')

In [13]:
df_final = df_final.replace(-1, np.nan)

In [14]:
#lambda x: pd.Series.mode(x) if (len(pd.Series.mode(x)) <= 1) else pd.Series.mode(x)[0]
# def agg_lambda(x):
#     print('Tipo', type(x))
    
#     print(x)

#     try:
#         moda = x.median().iloc[0]
#         print('mediana', moda)
#         print(type(moda))
#     except:
#         try:
#             moda = x.moda()[0]
#             print('moda - local', moda)
#         except:
#             print('ERRO')
#     return moda

#custom_sum = dd.Aggregation('custom_sum', agg_lambda, lambda x: x)

In [15]:
# def agg_func(array):
#     print(type(array), len(array))
#     try:
#         mode = pd.Series(array).mode()[0]
#     except:
#         try:
#             mode = pd.Series.mode(array)[0]
#         except:
#             mode = array
#             print('Erroooo')
    
#     print('ok')
#     return mode

In [16]:
# nan_sum_dask = dd.Aggregation('custom_nan_sum', 
#                               agg_func, 
#                               agg_func
#                               )

In [17]:
#df['c'] = [1, 2, 1, 1, 2]
#ddf = dd.from_pandas(df, 2)
# nunique = dd.Aggregation(
#     'nunique', 
#     lambda x: x, 
#     lambda y: y,
#     lambda z: z)
# custom_sum = dd.Aggregation('custom_sum', agg_func, agg_func)
# df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).aggregate({'TP_ESTADO_CIVIL': nan_sum_dask}).compute()
#df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).mean().compute()

In [18]:
# df_groups

In [19]:
# aggregations = {
#    'TP_ESTADO_CIVIL': 'mean',
#     'NU_NOTA_CN':'mean',
#     'NU_NOTA_CH': 'mean',
#     'NU_NOTA_LC': 'mean',
#     'NU_NOTA_MT': 'mean',
#   'NU_NOTA_COMP1': 'mean',
#   'NU_NOTA_COMP2': 'mean',
#   'NU_NOTA_COMP3': 'mean',
#   'NU_NOTA_COMP4': 'mean',
#   'NU_NOTA_COMP5': 'mean',
#   'NU_NOTA_REDACAO': 'mean'
# }

# df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).aggregate(aggregations).compute()
# df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [20]:
# df_groups

In [21]:
#imp_ec = SimpleImputer(strategy="most_frequent")
#imp_ec.fit(df_final[['TP_ESTADO_CIVIL']])

In [22]:
#df_final[['TP_ESTADO_CIVIL']] = imp_ec.transform(df_final[['TP_ESTADO_CIVIL']])

In [23]:
#df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).mean()
#df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [24]:
#df_groups.head()

In [25]:
# def add_null_values(x, df_groups):
#     if(sum(pd.isna(x)) > 0):
#         df_x = df_groups.query('TP_ESCOLA==@x.TP_ESCOLA & TP_ST_CONCLUSAO==@x.TP_ST_CONCLUSAO & SG_UF_RESIDENCIA==@x.SG_UF_RESIDENCIA')
    
#         for c in df_x.columns:
#             if(pd.isna(x[c]) > 0):
#                 x[c] = df_x[c].values[0]
#     return x

In [26]:
meta = dict(zip(df_final.columns, df_final._meta.dtypes))

In [27]:
# df_final = df_final.apply(lambda x: add_null_values(x, df_groups), axis=1, meta=meta)
# df_final = df_final.dropna() # Respostas do questionário nulas

In [28]:
# df_final.head()

In [29]:
df_final = df_final.dropna()

In [30]:
pd.DataFrame({'types': df_final.dtypes, 'missing': df_final.isna().sum()})

,types,missing
NU_INSCRICAO,float64,0
NU_ANO,float64,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,float64,0
TP_SEXO,object,0
TP_ST_CONCLUSAO,float64,0
TP_ANO_CONCLUIU,float64,0
TP_ESCOLA,float64,0
TP_ESTADO_CIVIL,int32,0


# Incluindo Status Socioeconômico

## Com base na renda per capita

### Respostas    
    - A	Nenhuma renda: 0.
    - B	Até  998,00:  Até 1.
    - C	De R$ 998,01 até R$ 1.497,00: 1 até 1,5.
    - D	De R$ 1.497,01 até R$ 1.996,00: 1,5 até 2.
    - E	De R$ 1.996,01 até R$ 2.495,00: 2 até 2,5.
    - F	De R$ 2.495,01 até R$ 2.994,00: 2,5 até 3.
    - G	De R$ 2.994,01 até R$ 3.992,00: 3 até 4.
    - H	De R$ 3.992,01 até R$ 4.990,00: 4 até 5.
    - I	De R$ 4.990,01 até R$ 5.988,00: 5 até 6.
    - J	De R$ 5.988,01 até R$ 6.986,00: 6 até 7.
    - K	De R$ 6.986,01 até R$ 7.984,00: 7 até 8.
    - L	De R$ 7.984,01 até R$ 8.982,00: 8 até 9.
    - M	De R$ 8.982,01 até R$ 9.980,00: 9 até 10.
    - N	De R$ 9.980,01 até R$ 11.976,00: 10 até 12.
    - O	De R$ 11.976,01 até R$ 14.970,00: 12 até 15.
    - P	De R$ 14.970,01 até R$ 19.960,00: 15 até 20.
    - Q	Mais de R$ 19.960,00: mais de 20.
    
    De 5000 participantes apenas 38 responderam como Q, então como seria inviável estimar a renda per capita consideramos que pertencem a classe A.


In [31]:
def add_class(x):
    income_dict = {'A': 0, 'B': 1, 'C': 1.5, 'D': 2, 'E': 2.5, 'F': 3, 'G': 4, 'H': 5, 'I': 6, 
                   'J': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 12, 'O': 15, 'P': 20}

    if(x['Q006'] == 'Q'): x['TP_SES_INCOME'] = 'A'
    else: 
        per_capita = income_dict[x['Q006']]/x['Q005']
        
        if (per_capita <= 1): x['TP_SES_INCOME'] = 'E'
        elif (per_capita <= 4):  x['TP_SES_INCOME'] = 'D'
        elif (per_capita <= 10): x['TP_SES_INCOME'] = 'C'
        else: x['TP_SES_INCOME'] = 'B'
        
    return x

In [32]:
meta['TP_SES_INCOME'] = 'O'
df_final['TP_SES_INCOME'] = ''

In [33]:
df_final = df_final.apply(lambda x: add_class(x), axis=1, meta=meta)

In [34]:
df_final.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ESTADO_CIVIL,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,TP_SES_INCOME
0,1.500000e+11,2015.0,Gravataí,RS,42.0,M,1.0,10.0,1.0,1.0,...,A,A,B,A,A,B,B,B,B,E
2,1.500000e+11,2015.0,Olinda,PE,22.0,M,4.0,0.0,1.0,1.0,...,A,A,A,A,A,C,B,B,A,E
4,1.500000e+11,2015.0,Teófilo Otoni,MG,18.0,M,1.0,2.0,1.0,0.0,...,A,B,B,A,A,C,B,B,B,E
5,1.500000e+11,2015.0,Canelinha,SC,19.0,M,1.0,3.0,1.0,0.0,...,A,B,D,A,B,D,B,C,B,E
6,1.500000e+11,2015.0,Guarulhos,SP,17.0,F,2.0,0.0,2.0,0.0,...,A,B,C,B,B,D,B,C,B,D


## Com base no sistema de pontos

    - Banheiros 0 3 7 10 14 - Q008
    - Empregados domésticos 0 3 7 10 13 - Q007
    - Automóveis 0 3 5 8 11 - Q010 
    - Microcomputador 0 3 6 8 11 - Q024
    - Lava louca 0 3 6 6 6 - Q017
    - Geladeira 0 2 3 5 5 - Q012
    - Freezer 0 2 4 6 6 - Q013
    - Lava roupa 0 2 4 6 6 - Q014
    - DVD 0 1 3 4 6 - Q020
    - Micro-ondas 0 2 4 4 4 - Q016
    - Motocicleta 0 1 3 3 3 - Q011
    - Secadora roupa 0 2 2 2 2 - Q015

    - Q019 - Na sua residência tem televisão em cores? 0 1 2 2 2
    - Q025 - Na sua residência tem acesso à Internet? 0 4


    -	Q001 ou Q002
    - Analfabeto / Fundamental I incompleto 0
    - Fundamental I completo / Fundamental II incompleto 1
    - Fundamental II completo / Médio incompleto 2
    - Médio completo / Superior incompleto 4
    - Superior completo 7

    - Água encanada 0 4
    - Rua pavimentada 0 2

In [35]:
with open("../datasets/sistema_pontos.json", encoding='utf-8') as json_:
    sistema_pontos = json.load(json_)

In [36]:
sistema_pontos

{'Q008': {'A': 0, 'B': 3, 'C': 7, 'D': 10, 'E': 14},
 'Q007': {'A': 0, 'B': 0, 'C': 0, 'D': 3},
 'Q010': {'A': 0, 'B': 3, 'C': 5, 'D': 8, 'E': 11},
 'Q024': {'A': 0, 'B': 3, 'C': 6, 'D': 8, 'E': 11},
 'Q017': {'A': 0, 'B': 3, 'C': 6, 'D': 6, 'E': 6},
 'Q012': {'A': 0, 'B': 2, 'C': 3, 'D': 5, 'E': 5},
 'Q013': {'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6},
 'Q014': {'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6},
 'Q020': {'A': 0, 'B': 1},
 'Q016': {'A': 0, 'B': 2, 'C': 4, 'D': 4, 'E': 4},
 'Q011': {'A': 0, 'B': 1, 'C': 3, 'D': 3, 'E': 3},
 'Q015': {'A': 0, 'B': 2, 'C': 2, 'D': 2, 'E': 2},
 'Q019': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
 'Q025': {'A': 0, 'B': 4},
 'Q001': {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 4, 'F': 7, 'G': 7, 'H': 0},
 'Q002': {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 4, 'F': 7, 'G': 7, 'H': 0}}

In [37]:
def add_class_point(x):
    points = 0 
    for index in x.index[25:]:
        if(index in sistema_pontos):
            if(index in ['Q001', 'Q002']):
                pm = sistema_pontos['Q001'][x['Q001']]
                pp = sistema_pontos['Q002'][x['Q002']]
                
                points += max(pm, pp)
            else:
                points += sistema_pontos[index][x[index]]
    
    if (points <= 16): x['TP_SES_POINTS'] = 'DE'
    elif (points <= 22):  x['TP_SES_POINTS'] = 'C2'
    elif (points <= 28): x['TP_SES_POINTS'] = 'C1'
    elif (points <= 37): x['TP_SES_POINTS'] = 'B2'
    elif (points <= 44): x['TP_SES_POINTS'] = 'B1'
    else: x['TP_SES_POINTS'] = 'A'
    
    return x

In [38]:
meta['TP_SES_POINTS'] = 'O'
df_final['TP_SES_POINTS'] = ''

In [39]:
df_final = df_final.apply(lambda x: add_class_point(x), axis=1, meta=meta)

In [40]:
df_final.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ESTADO_CIVIL,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,TP_SES_INCOME,TP_SES_POINTS
0,1.500000e+11,2015.0,Gravataí,RS,42.0,M,1.0,10.0,1.0,1.0,...,A,B,A,A,B,B,B,B,E,C2
2,1.500000e+11,2015.0,Olinda,PE,22.0,M,4.0,0.0,1.0,1.0,...,A,A,A,A,C,B,B,A,E,DE
4,1.500000e+11,2015.0,Teófilo Otoni,MG,18.0,M,1.0,2.0,1.0,0.0,...,B,B,A,A,C,B,B,B,E,C2
5,1.500000e+11,2015.0,Canelinha,SC,19.0,M,1.0,3.0,1.0,0.0,...,B,D,A,B,D,B,C,B,E,B1
6,1.500000e+11,2015.0,Guarulhos,SP,17.0,F,2.0,0.0,2.0,0.0,...,B,C,B,B,D,B,C,B,D,B1


In [41]:
%%time

# Save
file = '../datasets/integrated/integrated_datas_*.csv'
df_final.to_csv(file, index=False)

#z = zipfile.ZipFile('../datasets/integrated_data.zip', 'w', zipfile.ZIP_DEFLATED)
#z.write(file)    
#z.close()

C:\Users\mtbl\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


Wall time: 685 ms


['C:/Users/mtbl/Documents/Tassiane/SiDi/Projeto Educação/Enem/enem/datasets/integrated/integrated_datas_0.csv',
 'C:/Users/mtbl/Documents/Tassiane/SiDi/Projeto Educação/Enem/enem/datasets/integrated/integrated_datas_1.csv',
 'C:/Users/mtbl/Documents/Tassiane/SiDi/Projeto Educação/Enem/enem/datasets/integrated/integrated_datas_2.csv',
 'C:/Users/mtbl/Documents/Tassiane/SiDi/Projeto Educação/Enem/enem/datasets/integrated/integrated_datas_3.csv',
 'C:/Users/mtbl/Documents/Tassiane/SiDi/Projeto Educação/Enem/enem/datasets/integrated/integrated_datas_4.csv']

# Carregando o df_final

In [42]:
nums = [0, 1, 2, 3, 4]

In [43]:
df_test = pd.DataFrame([])

for n in nums:
    df = pd.read_csv(f'../datasets/integrated/integrated_datas_{n}.csv')
    df_test = pd.concat([df_test, df])

In [44]:
df_test

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ESTADO_CIVIL,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,TP_SES_INCOME,TP_SES_POINTS
0,1.500000e+11,2015.0,Gravataí,RS,42.0,M,1.0,10.0,1.0,1.0,...,A,B,A,A,B,B,B,B,E,C2
1,1.500000e+11,2015.0,Olinda,PE,22.0,M,4.0,0.0,1.0,1.0,...,A,A,A,A,C,B,B,A,E,DE
2,1.500000e+11,2015.0,Teófilo Otoni,MG,18.0,M,1.0,2.0,1.0,0.0,...,B,B,A,A,C,B,B,B,E,C2
3,1.500000e+11,2015.0,Canelinha,SC,19.0,M,1.0,3.0,1.0,0.0,...,B,D,A,B,D,B,C,B,E,B1
4,1.500000e+11,2015.0,Guarulhos,SP,17.0,F,2.0,0.0,2.0,0.0,...,B,C,B,B,D,B,C,B,D,B1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,1.900010e+11,2019.0,Santarém,PA,33.0,M,1.0,13.0,1.0,2.0,...,A,C,B,A,C,A,A,A,E,DE
678,1.900012e+11,2019.0,São Luís,MA,24.0,M,1.0,5.0,1.0,1.0,...,A,C,B,A,C,A,A,A,E,DE
679,1.900015e+11,2019.0,Bom Conselho,PE,18.0,F,1.0,1.0,1.0,1.0,...,A,B,A,A,C,A,A,B,E,C2
680,1.900019e+11,2019.0,Santa Tereza do Oeste,PR,18.0,F,2.0,0.0,2.0,1.0,...,B,B,A,A,D,B,B,B,E,B1


In [48]:
df_test['TP_SES_INCOME'].value_counts()

E    2708
D     587
C      39
A      35
Name: TP_SES_INCOME, dtype: int64

In [50]:
df_test['TP_SES_POINTS'].value_counts()

B2    759
DE    748
C2    730
C1    697
B1    228
A     207
Name: TP_SES_POINTS, dtype: int64

# Testes anos

In [ ]:
# def load_data_year_2015(year, cols):
#     file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
#     try:
#         df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=1000)
#     except:
#         df = pd.read_csv(file, encoding='cp1252', usecols=cols, nrows=1000)
    
#     return df

In [ ]:
# df_2014 = pd.read_csv('../../../microdados_anos/MICRODADOS_ENEM_2014.csv', encoding='cp1252', nrows=1000)
# df_2014.columns.values

In [ ]:
# len(['NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_CH', 'TP_PRESENCA_MT', 'NU_NOTA_MT', 'SG_UF_RESIDENCIA', 'TP_PRESENCA_CH', 
#  'NU_IDADE', 'TP_ANO_CONCLUIU', 'TP_PRESENCA_LC', 'TP_ST_CONCLUSAO', 'TP_PRESENCA_CN'])

In [ ]:
# df_2014 = df_2014.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  'NOTA_MT': 'NU_NOTA_MT',
#                         'IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
#                         'UF_RESIDENCIA': 'SG_UF_RESIDENCIA', 'IDADE': 'NU_IDADE', 'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
#                         'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})

In [ ]:
# df_2014[cols_used]

In [ ]:
# df_2015 = load_data_year(2015, cols_used)
# df_2015.columns.values

In [ ]:
# #lambda x: pd.Series.mode(x) if (len(pd.Series.mode(x)) <= 1) else pd.Series.mode(x)[0]
# def agg_lambda(x):
#     print('Teste', type(x))
#     print(x.mode)
#     mode = dd.Series.mode(x)
    
#     if (len(mode) <= 1):
#         return mode
#     else:
#         return mode[0]

In [ ]:
{
    'Q008': { 'A': 0, 'B': 3, 'C': 7, 'D': 10, 'E': 14 },
    'Q007': { 'A': 0, 'B': 0, 'C': 0, 'D': 3 },
    'Q010': { 'A': 0, 'B': 3, 'C': 5, 'D': 8, 'E': 11 },
    'Q024': { 'A': 0, 'B': 3, 'C': 6, 'D': 8, 'E': 11 },
    'Q017': { 'A': 0, 'B': 3, 'C': 6, 'D': 6, 'E': 6 },
    'Q012': { 'A': 0, 'B': 2, 'C': 3, 'D': 5, 'E': 5 },
    'Q013': { 'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6 },
    'Q014': { 'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 6 },
    'Q020': { 'A': 0, 'B': 1},
    'Q016': { 'A': 0, 'B': 2, 'C': 4, 'D': 4, 'E': 4 },
    'Q011': { 'A': 0, 'B': 1, 'C': 3, 'D': 3, 'E': 3 },
    'Q015': { 'A': 0, 'B': 2, 'C': 2, 'D': 2, 'E': 2 },
    'Q019': { 'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 3 },
    'Q025': { 'A': 0, 'B': 4 },
    'Q001': { 'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 4, 'F': 7, 'G': 7, 'H': 0},
    'Q002': { 'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 4, 'F': 7, 'G': 7, 'H': 0}
}